In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import requests
import shutil
import bs4
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
import pyautogui
import cv2
import time

In [27]:
PATH = "C:\chromedriver.exe"
driver = webdriver.Chrome(PATH)



#url that you want to scrape
link = 'https://www.tiktok.com/@ceritarumah_/video/7205460676901391643'

#open and maximize window
driver.get(link)
driver.maximize_window()

time.sleep(6) #nanti diganti tunggunya sampai captha nya nge load

#klik zoom
pyautogui.moveTo(1898, 57, duration=0.5)
pyautogui.click()
time.sleep(0.5)
pyautogui.moveTo(1836, 238, duration=0.5)
pyautogui.doubleClick()

#CAPTCHA SOLVER
#download image function
def down(image, xpath):
    #get link image
    captcha = driver.find_element (By.XPATH, xpath)
    src = captcha.get_attribute('src')

    #download image
    response = requests.get(src, stream=True)
    with open(image, 'wb') as out_file:
        shutil.copyfileobj(response.raw, out_file)
    del response


#download captcha image
down('captcha_img.jpeg', "//*[@id='captcha-verify-image']")
#download piece of captcha image
down('captcha_piece.jpeg', "//*[@id='tiktok-verify-ele']/div/div[2]/img[2]")

time.sleep(2)

#read img
img = cv2.imread('captcha_img.jpeg')
img_piece = cv2.imread('captcha_piece.jpeg')

#CONTOURS FUNCTION
def bin_img(image):
    img = cv2.imread(image)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    binary_img = cv2.threshold(img_gray, 242, 255, cv2.THRESH_BINARY_INV)[1]

    return binary_img

capt = bin_img('captcha_img.jpeg')
piece = bin_img('captcha_piece.jpeg')

#matching shape
match = cv2.matchTemplate(capt, piece, cv2.TM_CCOEFF_NORMED)

#check x, y in maximum value
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(match)

#width and height
w = piece.shape[1]
h = piece.shape[0]

#drag slider with mouse movement
time.sleep(2)

#gerakin mouse ke slider button
pyautogui.moveTo(782, 722, duration=1.5)

#drag mouse ke puzzle pece yang benar
pyautogui.dragRel((0.78*max_loc[0]), 0, button='left', duration=2)

time.sleep(5)

#exit login bar
ext = driver.find_element(By.XPATH, '//*[@id="login-modal"]/div[2]')
ext.click()

time.sleep(2)

#open profile
#profile = driver.find_element(By.XPATH, '//*[@id="main-content-video_detail"]/div/div[2]/div[1]/div[2]/div/a[2]/span[1]')
#profile.click()

#time.sleep(2)

C:\Users\user\AppData\Local\Temp\ipykernel_4644\107424534.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


### 2nd step testing

In [28]:
links = []#dibawah
title = []#dibawah
username = []
likes = []
views = []#dibawah
comment = []
share = []

#start and pause the video
pyautogui.moveTo(367, 997, duration=1)
pyautogui.click(button='left')
time.sleep(2)
pyautogui.click(button='left')
time.sleep(1)


#find username
xpath_username= '//*[@id="main-content-video_detail"]/div/div[2]/div[1]/div[2]/div/a[2]/span[1]'
link_username = driver.find_element(By.XPATH, xpath_username)
get_link_username = link_username.text
#find likes
xpath_likes = '//*[@id="main-content-video_detail"]/div/div[2]/div[1]/div[1]/div[1]/div[4]/div/button[1]/strong'
link_likes = driver.find_element(By.XPATH, xpath_likes)
get_link_likes = link_likes.text
#find comment
xpath_comment = '//*[@id="main-content-video_detail"]/div/div[2]/div[1]/div[1]/div[1]/div[4]/div/button[2]/strong'
link_comment = driver.find_element(By.XPATH, xpath_comment)
get_link_comment = link_comment.text
#find share
xpath_share = '//*[@id="main-content-video_detail"]/div/div[2]/div[1]/div[1]/div[1]/div[4]/div/button[3]/strong'
link_share = driver.find_element(By.XPATH, xpath_share)
get_link_share = link_share.text


#adding info to list
username.append(get_link_username)
likes.append(get_link_likes)
comment.append(get_link_comment)
share.append(get_link_share)

In [29]:
#open profile
profile = driver.find_element(By.XPATH, '//*[@id="main-content-video_detail"]/div/div[2]/div[1]/div[2]/div/a[2]/span[1]')
profile.click()

time.sleep(2)

#scroll to the bottom
previous_height = driver.execute_script('return document.body.scrollHeight')

while True:
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    time.sleep(3)
    new_height = driver.execute_script('return document.body.scrollHeight')
    
    if new_height == previous_height:
        break
    previous_height = new_height

#scroll to the top
driver.execute_script('window.scrollTo(0, 0)')

#take the html source
page_html = driver.page_source
page_soup = bs4.BeautifulSoup(page_html, 'html.parser')
#find all content
content = page_soup.find_all('div', {'class':"tiktok-x6y88p-DivItemContainerV2 e19c29qe7"})

len_content = len(content)
print(len_content)

241


In [30]:
for j in range(1, len_content+1):
    #find url
    xpath_url = f'//*[@id="main-content-others_homepage"]/div/div[2]/div[2]/div/div[{j}]/div[1]/div/div/a'
    link_url = driver.find_element(By.XPATH, xpath_url)
    get_link_url = link_url.get_attribute('href')

    if get_link_url != link:
        continue
    #find views
    xpath_views = f'//*[@id="main-content-others_homepage"]/div/div[2]/div[2]/div/div[{j}]/div[1]/div/div/a/div/div[2]/strong'
    link_views = driver.find_element(By.XPATH, xpath_views)
    get_link_views = link_views.text
    #find title
    xpath_title = f'//*[@id="main-content-others_homepage"]/div/div[2]/div[2]/div/div[{j}]/div[2]/a'
    link_title = driver.find_element(By.XPATH, xpath_title)
    get_link_title = link_title.get_attribute('title')

    #adding info
    links.append(get_link_url)
    title.append(get_link_title)
    views.append(get_link_views)

    break

#//*[@id="main-content-others_homepage"]/div/div[2]/div[2]/div/div[3]/div[1]/div/div/a
#//*[@id="main-content-others_homepage"]/div/div[2]/div[2]/div/div[{j}]/div[1]/div/div/a

#//*[@id="main-content-others_homepage"]/div/div[2]/div[2]/div/div[{j}]/div[1]/div/div/a
#//*[@id="main-content-others_homepage"]/div/div[2]/div[2]/div/div[4]/div[1]/div/div/a

In [ ]:
#//*[@id="main-content-others_homepage"]/div/div[2]/div[2]/div/div[1]/div[2]/a
#//*[@id="main-content-others_homepage"]/div/div[2]/div[2]/div/div[2]/div[2]/a

In [31]:
print(links)
print(title)
print(username)
print(likes)
print(views)
print(comment)
print(share)

['https://www.tiktok.com/@ceritarumah_/video/7205460676901391643']
['Bismillah #rumahminimalis #rumahmilenial #rumah1lantai ']
['ceritarumah_']
['376K']
['5.7M']
['3427']
['39.1K']


In [6]:
time.sleep(4)
pyautogui.displayMousePosition()

Press Ctrl-C to quit.
X: 1836 Y:  238 RGB: ( 32,  33,  3X: 1836 Y:  238 RGB: ( 32,  33,  3X: 1836 Y:  238 RGB: ( 32,  33,  3X: 1836 Y:  238 RGB: ( 32,  33,  3X: 1836 Y:  238 RGB: ( 32,  33,  3X: 1836 Y:  238 RGB: ( 32,  33,  3X: 1836 Y:  238 RGB: ( 32,  33,  3X: 1836 Y:  238 RGB: ( 32,  33,  3X: 1836 Y:  238 RGB: ( 32,  33,  3X: 1836 Y:  238 RGB: ( 32,  33,  3X: 1836 Y:  238 RGB: ( 32,  33,  3X: 1836 Y:  238 RGB: ( 32,  33,  3X: 1836 Y:  238 RGB: ( 32,  33,  3X: 1836 Y:  238 RGB: ( 32,  33,  3X: 1836 Y:  238 RGB: ( 32,  33,  3X: 1836 Y:  238 RGB: ( 32,  33,  3X: 1836 Y:  238 RGB: ( 32,  33,  3X: 1836 Y:  238 RGB: ( 32,  33,  3X: 1836 Y:  238 RGB: ( 32,  33,  3X: 1836 Y:  238 RGB: ( 32,  33,  3X: 1836 Y:  238 RGB: ( 32,  33,  3X: 1836 Y:  238 RGB: ( 32,  33,  3X: 1836 Y:  238 RGB: ( 32,  33,  3X: 1836 Y:  238 RGB: ( 32,  33,  3X: 1836 Y:  238 RGB: ( 32,  33,  3X: 1836 Y:  238 RGB: ( 32,  33,  3X: 1836 Y:  238 RGB: ( 32,  33,  3X: 1836 Y:  238 RGB: ( 32,  33,  3X: 1836 Y:  238 RGB: ( 32,

In [34]:
#start and pause the video
time.sleep(2)
pyautogui.moveTo(367, 1002, duration=1)
pyautogui.click()
time.sleep(2)
pyautogui.click()

# coret2an

In [20]:
#untuk membersihkan link yg di profile sebelum disamakan dengan link yg kita punya
c = 'https://www.tiktok.com/@dndalstri._/video/7233570284572249370?is_from_webapp=1&sender_device=pc&web_id=7205425525594031618'
c[:c.find('?')]

'https://www.tiktok.com/@dndalstri._/video/7233570284572249370'